

 # **Ejercicios prácticos de acceso vía Entrez/E-utilities**

In [2]:
from typing import Optional
import json
import requests
import jsonpath_ng.ext as jp
import pandas as pd

EInfo = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/einfo.fcgi"
ESearch = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
ESummary = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi"
EFetch = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"



# 1. **EInfo: buscar todas las bases de datos disponibles**
 Genera una tabla con:
 - Nombre de la base de datos
 - Descripción
 - Número de registros
 - Fecha de última Actualización.

In [ ]:
def get_einfo(db:Optional[str]= None) -> dict:
    params = { "retmode": "json" }
    if db:
        params.update({"db": db})
    response = requests.get(EInfo, params=params, timeout=10)
    response.raise_for_status()
    return response.json()

In [ ]:
all_dbs = [ x.value for x in jp.parse("$..dblist[*]").find(get_einfo()) ]

In [ ]:
all_dbs_data = []
for current_db in all_dbs:
    data = get_einfo(current_db)
    dbname = jp.parse("$..dbname").find(data)[0].value
    dbdescription = jp.parse("$..description").find(data)[0].value
    dbcount = jp.parse("$..count").find(data)[0].value
    dblastupdate = jp.parse("$..lastupdate").find(data)[0].value
    all_dbs_data.append([dbname, dbdescription, dbcount, dblastupdate])

all_dbs_table = pd.DataFrame(
    all_dbs_data,
    columns=["name", "dbdescription", "count", "lastupadte"]
)

In [33]:
display(all_dbs_table)

,name,dbdescription,count,lastupadte
0,pubmed,PubMed bibliographic record,38714932,2025/04/23 19:05
1,protein,Protein sequence record,1446855351,2025/04/22 18:18
2,nuccore,Core Nucleotide db,657965355,2025/04/24 00:03
3,ipg,Identical Protein Groups DB,943145763,2025/04/23 23:03
4,nuccore,Core Nucleotide db,657965355,2025/04/24 00:03
5,structure,Three-dimensional molecular model,234798,2025/04/17 01:10
6,genome,"Genomic sequences, contigs, and maps",88333,2024/11/18 13:03
7,annotinfo,Annotinfo Database,2281,2025/04/23 15:04
8,assembly,Genome Assembly Database,3065819,2025/04/23 15:43
9,bioproject,BioProject Database,883466,2025/04/23 15:46




 ## 2. **Buscar el ID de una secuencia de GenBank por nombre**
 Usa `esearch` para encontrar el **ID** de la secuencia del gen **BRCA1
 humano** en GenBank.

In [34]:
def eSearch(term: str, db: str) -> dict:
    params = { "retmode": "json", "db":db, "term": term }
    response = requests.get(ESearch, params=params, timeout=10)
    response.raise_for_status()
    return response.json()

In [ ]:
term = 'BRCA1[Gene Name] AND "Homo sapiens"[Organism] AND BRCA1[Title] AND (biomol_genomic[PROP] AND refseq[filter])'
data = eSearch(term, "nuccore")

n_results = jp.parse("$..count").find(data)[0].value
print(f"Se encontraron: {n_results} registros.")
print("El id es:", jp.parse("$..idlist[0]").find(data)[0].value)

Se encontraron: 3 registros.
El id es: 262359905




 ## 3. **Obtener el archivo GenBank completo para un Accession ID**
 Usa `efetch` para descargar el archivo GenBank (`retmode=gb`) del gen
 **COX1** de *Homo sapiens*.

In [64]:
def efetch(id: str, db: str, format: str):
    if format == "genbank":
        retmode = "text"
        rettype = "gb"
    if format == "fasta":
        retmode = "text"
        rettype = "fasta"
    params = {
        "db": db,
        "id": id,
        "retmode": retmode,
        "rettype": rettype
    }
    response = requests.get(EFetch, params=params, timeout=10)
    response.raise_for_status()
    return response.content.decode()

In [55]:
# Hay dos genes COX1 humanos, uno en cromosoma 9 y otro en mitocondria
# La cromosomica se llama oficialmente PTGS1 y la mitocondrial MT-CO1

term = 'COX1[Gene Name] AND "Homo sapiens"[Organism] AND PTGS1[Title] AND (biomol_genomic[PROP] AND refseq[filter])'
data = eSearch(term, "nuccore")

n_results = jp.parse("$..count").find(data)[0].value
print(f"Se encontraron: {n_results} registros.")
current_id = jp.parse("$..idlist[0]").find(data)[0].value
print("El id es:", current_id)

Se encontraron: 1 registros.
El id es: 401709987


In [61]:
gb = efetch(current_id, "nuccore", "genbank")

with open("cox1.genbank", "w", encoding="utf-8") as fout:
    fout.write(gb)



 ## 4. **Descargar la secuencia en formato FASTA**
 Busca y descarga la secuencia en formato **FASTA** del gen **16S rRNA** de
 *Escherichia coli*.

In [63]:
term = '"16S rRNA"[Gene Name] AND "Escherichia coli"[Organism] AND 16S[Title] NOT partial[title] AND biomol_genomic[PROP]'

search = eSearch(term, "nuccore")

n_results = jp.parse("$..count").find(search)[0].value
print(f"Se encontraron: {n_results} registros.")
current_ids = [x.value for x in jp.parse("$..idlist[*]").find(search) ]
print("Los Ids son:", current_ids)

Se encontraron: 11 registros.
Los Ids son: ['82617088', '82617077', '82617066', '38091817', '32127561', '1865789', '1865788', '1240024', '1240023', '1240022', '1240021']


In [ ]:
result = efetch(",".join(current_ids), db="nuccore", format="fasta")
with open("16s.fasta", "w", encoding="utf8") as fout:
    fout.write(result)




 ## 5. **Listar todos los Accessions de un organismo**
 Usa `esearch` + `efetch` para listar los **primeros 20 Accessions** de
 secuencias de *Arabidopsis thaliana* en RefSeq.

In [66]:
term = 'Arabidopsis thaliana[Organism] and srcdb_refseq'

search = eSearch(term, "nuccore")

for c_id in jp.parse("$..idlist[*]").find(search):
    print(f"Id: {c_id.value}")

Id: 240256493
Id: 240256243
Id: 240255695
Id: 240254678
Id: 1373662007
Id: 7525012
Id: 2319564279
Id: 1162502463
Id: 1162502461
Id: 1162502459
Id: 1162502457
Id: 1162502455
Id: 1162502453
Id: 1162502451
Id: 1162502449
Id: 1162502447
Id: 1162502442
Id: 1162502440
Id: 1162502438
Id: 1162502436




 ## 6. **Obtener los metadatos de una secuencia**
 Con `esummary`, extrae información como el título, longitud y fecha de
 publicación para una secuencia dada.

In [ ]:
def eSummary(id:str, db:str) -> dict:
    params = {
        "id": id,
        "retmode": "json",
        "db": db
    }
    response = requests.get(ESummary, params=params, timeout=20)
    response.raise_for_status()
    return response.json()

In [70]:
def get_metadata(id:str):
    data = eSummary(id, "nuccore")
    title = jp.parse("$..title").find(data)[0].value
    sequence_length = jp.parse("$..slen").find(data)[0].value
    create_date = jp.parse("$..createdate").find(data)[0].value
    return {
        "title": title,
        "sequence_length": sequence_length,
        "create_date": create_date
    }

In [71]:
get_metadata("1162502436")

{'title': 'Arabidopsis thaliana uncharacterized protein (AT3G44766), mRNA',
 'sequence_length': 241,
 'create_date': '2017/03/20'}



 ## 7. **Buscar genes por palabra clave en la definición**
 Busca secuencias que contengan la palabra "photosystem II" en su definición y
 recupera los primeros 10 Accessions.

In [ ]:
# La definición en un archivo Genbank es lo que muestra como título.
term = '"photosystem II"[Title]'
search = eSearch(term, "nuccore")


n_results = jp.parse("$..count").find(search)[0].value
print(f"Se encontraron: {n_results} registros.")

current_ids = [x.value for x in jp.parse("$..idlist[*]").find(search)][:10]
print("Los 10 primeros Ids son:", current_ids)


Se encontraron: 98987 registros.
Los Ids son: ['2955799386', '2958074215', '2958074213', '2958074097', '2958051237', '2958047185', '2958034616', '2958029509', '2958028539', '2958021620']


In [ ]:

summaries = eSummary(",".join(current_ids), "nuccore")


summaries

{'header': {'type': 'esummary', 'version': '0.3'},
 'result': {'uids': ['2955799386',
   '2958074215',
   '2958074213',
   '2958074097',
   '2958051237',
   '2958047185',
   '2958034616',
   '2958029509',
   '2958028539',
   '2958021620'],
  '2955799386': {'uid': '2955799386',
   'term': '2955799386',
   'caption': 'PP476488',
   'title': 'Tryblionella hungarica voucher SZCZE683 photosystem II CP43 protein (psbC) gene, partial cds; chloroplast',
   'extra': 'gi|2955799386|gb|PP476488.1|',
   'gi': 2955799386,
   'createdate': '2025/04/19',
   'updatedate': '2025/04/19',
   'flags': '',
   'taxid': 2267605,
   'slen': 1050,
   'biomol': 'genomic',
   'moltype': 'dna',
   'topology': 'linear',
   'sourcedb': 'insd',
   'segsetsize': '',
   'projectid': '0',
   'genome': 'chloroplast',
   'subtype': 'specimen_voucher|country|isolation_source|lat_lon|collection_date|collected_by',
   'subname': 'SZCZE683|Poland|Pelczyska pond|51.9766 N 19.2381 E|2014-10|Rafal M. Olszynski',
   'assemblygi'

In [95]:
id_acc = []
for c_id in current_ids:
    query = f'$.result.["{c_id}"].accessionversion'
    accession = jp.parse(query).find(summaries)[0].value
    id_acc.append([c_id, accession])

id_acc_table = pd.DataFrame(
    id_acc,
    columns=["ID", "Accession"]
)
display(id_acc_table)

,ID,Accession
0,2955799386,PP476488.1
1,2958074215,XM_073272086.1
2,2958074213,XM_073272085.1
3,2958074097,XM_073272031.1
4,2958051237,XM_073302047.1
5,2958047185,XM_073300003.1
6,2958034616,XM_073293215.1
7,2958029509,XM_073290826.1
8,2958028539,XM_073290339.1
9,2958021620,XM_073286719.1




 ## 8. **Filtrar por tipo de molécula**
 Busca secuencias de ARN mensajero (**mRNA**) en *Mus musculus* relacionadas
 con "interleukin".

In [ ]:
term = ""



 ## 9. **Obtener la traducción de una proteína a partir del CDS**
  Extrae el campo `/translation` de una entrada GenBank que contenga una
  secuencia codificante (CDS).



 ## 10 **Buscar todos los genomas completos de un organismo**
  Encuentra todas las entradas de genomas **completos** de *Mycobacterium
  tuberculosis*.



 ## 11. **Extraer estadísticas de longitud**
  Descarga 100 secuencias de genes de *Saccharomyces cerevisiae* y calcula la
  distribución de longitudes.



 ## 12. **Buscar entradas con anotaciones específicas**
 Buscar 10 entradas que tengan el qualifier
 `/product="cytochrome c oxidase subunit I"`
  y guarda sus FASTA en archivos separados.



 ## 13. **Número de registros en el tiempo**
 Usa `esearch` para contar el número de registros en GenBank de secuencias de
 genomas completos de HIV1 por mes desde 1986 hasta la actualidad. Elimina
 todas aquellas que en el título diga "UNVERIFIED".



 ## 14. **Comparar Tabla de CDS**
 Descarga las tablas de features para todos genomas mitocondriales completos
 de C. elegans y compara sus CDS. ¿Cuántos son iguales?
 ¿Cuántos son diferentes?

## 15. **Recuperar genes en region cromosómica**

Recupera los nombre de todos los genes que se encuentran en el cromosoma 10,
entre las posiciones 100000 y 1000000 en humanos y en ratón.



## 16. **Recuperar variantes**

Recuperar cuantas variantes existen para el gen FUS humano en la base de datos
dbVar, que tengan una frecuencia alélica global menor al 1%.

